# ***CloudPhysician***
# ***THE VITAL EXTRACTION CHALLENGE***

### ***Extracting Patient Vitals from the given image of vital sign monitor using ML model, Image Processing and Optical Character Recognition.***

## ***About the data***

- The data consists of various images of health monitors. The classification dataset of the 4 types of monitors was used. Python code was used to randomly shuffle the images and pick 800 for training, and 200 for validation.
The script created the following folder format:
dataset - images - (train + val folders)
        - labels - (train + val folders)
All the image and label filenames are indexed by numbers. The various label files contain annotations for the images. An example annotation is:
```
0 0.78359375 0.14375 0.065625 0.09583333333333334
3 0.89921875 0.25416666666666665 0.0453125 0.06944444444444445
4 0.794140625 0.5027777777777778 0.08515625 0.10277777777777777
5 0.778515625 0.6208333333333333 0.05546875 0.10277777777777777
6 0.34765625 0.37083333333333335 0.678125 0.09166666666666666
7 0.34765625 0.6201388888888889 0.6734375 0.08472222222222223
8 0.348046875 0.5090277777777777 0.67578125 0.10694444444444444
```

- The first index is the label, and the 4 indices that follow are the normalized bounding box coordinates.
The labels correspond to the indices of the matrix `['HR','SBP','DBP','MAP','SPO2','RR','HR_W','RR_W','SPO2_W']`

- A dataset.yaml file is used to configure the model. It's format is:

```
path: dataset
train: images/train
val: images/val

nc: 9

names: `['HR','SBP','DBP','MAP','SPO2','RR','HR_W','RR_W','SPO2_W']`

```
## ***Getting Started***

To use the model, just run the corresponding cells in the Jupyter notebook. The cells will download the pretrained model from GitHub, and use it to generate the bounding boxes which will later be used for OCR detection.

## ***Neural Network Architecture***

The bounding boxes will also be generated using the yolov5 model:



***Why this architecture?***

YOLOv5 (You Only Look Once version 5) is an object detection model used in computer vision tasks. It is a fast and accurate model that can perform object detection in real-time. YOLOv5 is the latest version in the YOLO series and is based on a single shot multi-box detection (SSD) architecture. It uses anchor boxes to predict bounding boxes for objects in an image and assigns a class label to each bounding box. YOLOv5 is known for its ability to detect objects in real-time and its relatively low computational requirements compared to other object detection models.





Other architectures like the Faster-RCNN and SSD were tried, but the YoloV5 model was found to give the best (and fastest) results. Yolov5 allows us to use a state of the art architecture to quickly detect objects. The model was trained for the problem specification by using the custom dataset.

## ***Training the model***
The model was trained for 30 epochs and the following table was obtained:


|Class   |  Images | Instances  |     P    |      R    |  mAP50   |  mAP50-95: 100% 10/10  |
| ------ | ------- | ---------- | -------- | --------- | -------- | ---------------------- |
|all     |   300   |    2161    |  0.783   |   0.957   |   0.837  |      0.669             |
|HR      |   300   |     300    |  0.985   |       1   |   0.995  |      0.886             |
|SBP     |   300   |     273    |      1   |   0.867   |   0.989  |      0.842             |
|DBP     |   300   |     269    |  0.757   |       1   |   0.985  |      0.816             |
|MAP     |   300   |     271    |   0.98   |       1   |   0.995  |      0.718             |
|SPO2    |   300   |     281    |  0.762   |       1   |   0.988  |       0.88             |
|RR      |   300   |     298    |  0.988   |   0.835   |    0.98  |      0.829             |
|HR_W    |   300   |     163    |  0.539   |   0.994   |    0.54  |      0.379             |
|RR_W    |   300   |     160    |  0.541   |   0.988   |   0.573  |      0.342             |
|SPO2_W  |   300   |     146    |  0.492   |   0.932   |   0.494  |       0.33             |



## ***Output of ML Model***

Now, the best model was used to create bounding boxes for the various classes:<br>

It outputs two matrices:
- 1 dimensional matrix containing the labels of the predicted classes
- 2 dimensional matrix, containing the bounding box coordinates (normalised x and y, width and height) and confidence for the predicted labels
The predicted bounding boxes can also be displayed using `results.show()`

The two matrices are processed and used later for OCR detection.

## ***Processing of Image***

- Using the bounding box coordinates image is cropped.
- Cropped Image is then denoised using several denoising techniques and thresholded.
- The threshold value chosen is the optimal value where we can binarize the image without losing the text.
- Processed Image is then passed through OCR to predict the text in the image.
- All the predicted value is reported in the form of python dictionary format.

## ***OCR Configuration***

```
r"--psm 8 --oem 3 -c tessedit_char_whitelist=0123456789/"
```

This is a command-line argument for the Tesseract OCR engine. The arguments are as follows:

- `--psm 8` : Page Segmentation Mode 8 is used to recognize a single digit as an instance of the character class.

- `--oem 3` : The OCR Engine Mode 3 is used, which is a deep learning based recognition system.

- `-c tessedit_char_whitelist=0123456789/` : The character whitelist is set to the numbers 0 to 9, allowing the OCR engine to recognize only these characters.

The command line options are used to configure the behavior of the Tesseract OCR engine for optimal recognition of a specific type of text, in this case, single digits.

## ***HR Wave Digitization***

- The YoloV5 model is predicting the HR Wave and returning the coordinates of the bounding box.
- After that the HR bounding box is cropped to give an image containing the HR for digital ECG analysis.
- The function `digitalECG(path)` first reads an image, scales it, performs noise reduction and thresholding on it to obtain a binary image.
- Then it finds the R peaks in the ECG signal by computing the mean, mode, and R_central_difference of the digitized ECG signal.
- Finally, it plots the digital ECG signal, the thresholded image, and the original image.

# ***Explaining Functions Used***

## ***cropImage(img, normalized_coordinates)***
- This function crops an image (2D numpy array `img`) using the normalized coordinates specified in `normalized_coordinates` (a list of 4 values, each representing a fraction of the total image size).
- The function first calculates the width and height of the image using `img.shape[1]` and `img.shape[0]`, respectively.
- Then it extracts a rectangular region of the image, defined by the top-left and bottom-right corners, which are computed by scaling the normalized coordinates by the image size.
- The cropped region is returned as a new 2D numpy array.

## ***crop_Image(img, normalized_coordinates)***
- It is function similar to `cropImage`. It is specifically used to crop image of vital values by increasing the margin of cropped image so that edges of the numbers are not cut and is recognized perfectly for OCR.

## ***text_r(img)***
It is a function that performs optical character recognition (OCR) on an input image "img". The function does the following steps:

- Defines the OCR configuration using the "my_config" variable, which specifies that only digits and "/" should be recognized, and sets the Page Segmentation Mode (psm) to 8 and the OCR Engine Mode (oem) to 3.

- Removes noise from the image using the fastNlMeansDenoisingColored function from OpenCV library.
  
- Converts the image to binary format using the threshold function from OpenCV library.

- Converts the binary image to grayscale using the cvtColor function from OpenCV library.

- Uses the image_to_string function from the Tesseract OCR library to perform OCR on the grayscale image, and returns the recognized text.

## ***hr_wave_detection(img, labels, coords)***
This function takes an image, an array of labels and an array of coordinates as inputs. The function performs the following steps:

- Loops over each label in the input "labels" array.
- If the label is equal to 6, it calls the "crop_image" function to crop the image using the corresponding coordinates in the "coords" array.
- Calls the "cv2_imshow" function to display the cropped image, which is supposed to be the HR waveform.

## ***ocr(img, complete_coords, labels)***
- This function performs optical character recognition (OCR) on an input image `img` using `complete_coords` and `labels` as parameters.
- It creates an output dictionary `output_dict` and uses the reference list `reference` which has labels for the extracted data.
- The function crops the input image using the `crop_image` function and performs text recognition on the cropped image using the `text_r` function.
- The extracted text is stored in the `output_dict` with its corresponding label from the `reference` list.
- The function then returns the `output_dict` after all 6 labels have been processed.

## ***inference(image_path:str)***
It is a function that takes an image file path as an input and returns the result of OCR. The function performs the following steps:

- Calls the "monitor_model" function on the input image to get the locations of text regions in the image.
- Calls the "cropImage" function to crop the image based on the locations of the text regions.
- Calls the "vitals_model" function on the cropped image to get updated locations of the text regions.
- Calls the "ocr" function on the cropped image and the updated text region locations to perform OCR on the image.
- Returns the result of OCR.

In [ ]:
!python -m pip install --upgrade pip
!pip install pytesseract
!pip install tensorflow==2.3.1
!sudo apt-get install tesseract-ocr
!pip install tensorboard==2.4.1
!pip install torch
!pip uninstall PIL
!pip uninstall Pillow
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0r

In [ ]:
%matplotlib inline
import pytesseract as tess
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import torch
from IPython.display import Image
import statistics as st
import matplotlib.pyplot as plt

In [ ]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
!git clone https://github.com/FaizanRasoolQ/VitalSIgnExtraction.git

fatal: destination path 'VitalSIgnExtraction' already exists and is not an empty directory.


In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [ ]:
monitor_model = torch.hub.load('/content/yolov5/', 'custom', path='/content/VitalSIgnExtraction/monitor_best.pt', source='local')

YOLOv5 🚀 v7.0-128-gb96f35c Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
def cropImage(img, normalized_coordinates):
  img_width, img_height = img.shape[1],img.shape[0]
  return img[int(normalized_coordinates[0][1]*img_height):int(normalized_coordinates[0][3]*img_height), int(normalized_coordinates[0][0]*img_width):int(normalized_coordinates[0][2]*img_width)]


In [ ]:
vitals_model = torch.hub.load('/content/yolov5', 'custom', path='/content/VitalSIgnExtraction/vitalsign_best.pt', source='local')

YOLOv5 🚀 v7.0-128-gb96f35c Python-3.9.16 torch-1.13.1+cu116 CPU

Fusing layers... 
Model summary: 157 layers, 7034398 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [ ]:
def text_r(img):
    my_config = r"--psm 11 --oem 3 -c tessedit_char_whitelist=0123456789"
    noise_reduced = cv.fastNlMeansDenoisingColored(img, None, 5, 10, 7, 21)
    thresh, t_img = cv.threshold(noise_reduced, 200, 255, cv.THRESH_BINARY_INV)
    t_img = cv.cvtColor(t_img, cv.COLOR_BGR2GRAY)
    height, width, chaneels = img.shape
    text = tess.image_to_string(t_img, config= my_config)
    return text

In [ ]:
def crop_Image(img, normalized_coordinates):
    margin = 0.16
    new_img = img[(int(normalized_coordinates[1]*img.shape[0])):(int(normalized_coordinates[3]*img.shape[0])), int(normalized_coordinates[0]*img.shape[1]):int(normalized_coordinates[2]*img.shape[1])]
    return img[(int(normalized_coordinates[1]*img.shape[0]-margin*new_img.shape[0])):(int(normalized_coordinates[3]*img.shape[0]+margin*new_img.shape[0])), int(normalized_coordinates[0]*img.shape[1]-margin*new_img.shape[1]):int(normalized_coordinates[2]*img.shape[1]+margin*new_img.shape[1])]

In [ ]:
def hr_wave_detection(img):
  result=monitor_model(img)
  labels, cord_thres = result.xyxyn[0][:, -1].cpu().numpy(), result.xyxyn[0][:, :-1].cpu().numpy()
  new_image = cropImage(img, cord_thres)
  vital_results=vitals_model(new_image)
  labels, cord_thres = vital_results.xyxyn[0][:, -1].cpu().numpy(), vital_results.xyxyn[0][:, :-1].cpu().numpy()
  #vital_results.show()
  for i in range(len(labels)):
    if(labels[i] == 6):
      hr_wave = crop_Image(new_image, cord_thres[i])
      cv2_imshow(hr_wave)
      return hr_wave

In [ ]:
!pip install -q protobuf-compiler python-pil python-lxml
!pip install -q paddlepaddle
!pip install -q "paddleocr>=2.0.1"

ERROR: Could not find a version that satisfies the requirement python-pil (from versions: none)
ERROR: No matching distribution found for python-pil
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.1/383.1 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.9/445.9 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.9/305.9 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.2/608.2 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 79.8 MB/s eta 0:00:00
    

In [ ]:
from paddleocr import PaddleOCR,draw_ocr
ocr = PaddleOCR(use_angel_cls= True, lang = 'en')

In [ ]:
def text_paddle(img):
  result = ocr.ocr(img, cls=True)
  if(len(result[0]) != 0):
    return result[0][0][1][0]
  else:
    return 'ud'

In [ ]:
def paddle_ocr(img, complete_coords, labels, path_str):
  visited = np.zeros(6)
  refernce = ["HR", "SBP", "DBP", "MAP", "SPO2", "RR"]
  #monitor are the coords of the first out_put
  output_dict = {}
  j = 0
  num = 0
  for i in range(len(labels)):
      if j == 6:
          break
      elif labels[i] < 6 and visited[int(labels[i])] == 0:
              visited[int(labels[i])] = 1
              ind_img = crop_Image(img, complete_coords[i])
              cv2_imshow(ind_img)
              #cv.imwrite('/content/drive/MyDrive/hundred/{}/{}'.format(path_str, num), ind_img)
              #cv2_imshow(ind_img)
              text = text_paddle(ind_img)
              if len(text)!= 0:
                #print(text[0])
                if text[0]!='\x0c':
                  output_dict[refernce[int(labels[i])]] = text
              j+=1
      num +=1
  return output_dict

In [ ]:
def ocr_shit(img, complete_coords, labels):
  visited = np.zeros(6)
  refernce = ["HR", "SBP", "DBP", "MAP", "SPO2", "RR"]
  #monitor are the coords of the first out_put
  output_dict = {}
  j = 0
  for i in range(len(labels)):

      if j == 6:
          break
      elif labels[i] < 6 and visited[int(labels[i])] == 0:
              visited[int(labels[i])] = 1
              ind_img = crop_Image(img, complete_coords[i])
              cv2_imshow(ind_img)
              text = text_r(ind_img)
              text = text.split('\n')
              if len(text)!= 0:
                #print(text[0])
                if text[0]!='\x0c':
                  output_dict[refernce[int(labels[i])]] = text[0]
              j+=1

  return output_dict

In [ ]:
def inference(image_path:str):
  img=cv.imread(image_path)
  result=monitor_model(img)
  labels, cord_thres = result.xyxyn[0][:, -1].cpu().numpy(), result.xyxyn[0][:, :-1].cpu().numpy()
  #result.show()
  new_image = cropImage(img, cord_thres)
  vital_results=vitals_model(new_image)
  labels, cord_thres = vital_results.xyxyn[0][:, -1].cpu().numpy(), vital_results.xyxyn[0][:, :-1].cpu().numpy()
  #vital_results.show()
  result = paddle_ocr(new_image,cord_thres,labels, image_path)
  return result

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/hundred'

/content/drive/MyDrive/hundred


In [ ]:
import os
l_t = os.listdir('/content/drive/MyDrive/hundred')
outputs = []
# for i in range(10):
#   d = inference(l_t[i])
#   outputs.append(d)

In [ ]:
print(len(l_t))

102


In [ ]:
%pwd

'/content/drive/MyDrive/hundred'

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame()
df['image_path'] = np.nan
df['label'] = np.nan
df['text'] = np.nan

In [ ]:
def create_dataset(image_path, df):
  img=cv.imread(image_path)
  result=monitor_model(img)
  #result.show()
  labels, cord_thres_m = result.xyxyn[0][:, -1].cpu().numpy(), result.xyxyn[0][:, :-1].cpu().numpy()
  new_image = cropImage(img, cord_thres_m)
  #cv2_imshow(new_image)
  vital_results=vitals_model(new_image)
  labels, cord_thres = vital_results.xyxyn[0][:, -1].cpu().numpy(), vital_results.xyxyn[0][:, :-1].cpu().numpy()
  #print(cord_thres)
  #print(labels)
  for i in range(len(labels)):
    if(labels[i] < 6):
      ind_img = crop_Image(new_image, cord_thres[i])
      #cv2_imshow(ind_img)
      te = text_paddle(ind_img)
      print(te)
      df = df.append({'image_path': "/content/drive/MyDrive/hundred/Croped_images/{}_{}.jpg".format(image_path[:2], i), 'label': labels[i], 'text': te}, ignore_index=True)
      cv.imwrite("/content/drive/MyDrive/hundred/Croped_images/{}_{}.jpg".format(image_path[:2], i), ind_img)
  df.to_csv('data_{}.csv'.format(image_path[:2]), index=False)

In [ ]:
create_dataset(l_t[0], df)

In [ ]:
print(df)

Empty DataFrame
Columns: [image_path, label, text]
Index: []


In [ ]:
for l in range(2,10):
  create_dataset(l_t[l], df)

In [ ]:
print(l_t.sort)
print(l_t)

<built-in method sort of list object at 0x7f109c2a7940>
['.ipynb_checkpoints', '0.jpg', '1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '3.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '4.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '5.jpg', '50.jpg', '51.jpg', '52.jpg', '53.jpg', '54.jpg', '55.jpg', '56.jpg', '57.jpg', '58.jpg', '59.jpg', '6.jpg', '60.jpg', '61.jpg', '62.jpg', '63.jpg', '64.jpg', '65.jpg', '66.jpg', '67.jpg', '68.jpg', '69.jpg', '7.jpg', '70.jpg', '71.jpg', '72.jpg', '73.jpg', '74.jpg', '75.jpg', '76.jpg', '77.jpg', '78.jpg', '79.jpg', '8.jpg', '80.jpg', '81.jpg', '82.jpg', '83.jpg', '84.jpg', '85.jpg', '86.jpg', '87.jpg', '88.jpg', '89.jpg', '9.jpg', '90.jpg', '91.jpg', '92.jpg', '

In [ ]:
%tb

In [ ]:
import matplotlib.pyplot as plt
def digitalECG(img):
  #Scaling Image
  scale_percent = 300 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  imgr=cv.resize(img, dim, interpolation = cv.INTER_CUBIC)
  #Noise Reduction
  noise_reduced = cv.fastNlMeansDenoisingColored(imgr, None, 5, 10, 7, 21)
  noise_reduced = cv.cvtColor(noise_reduced, cv.COLOR_BGR2GRAY)
  #Thresholding
  thresh, t_img = cv.threshold(noise_reduced, 175, 255, cv.THRESH_BINARY)
  y,x=t_img.shape
  px = 1/plt.rcParams['figure.dpi']  # pixel in inches
  digitized=np.zeros(x)
  for i in range(x):
      for j in range(y-1,-1,-1):
        if t_img[j,i]==255:
          digitized[i]=j
          break
  min=np.min(digitized[np.nonzero(digitized)])
  max=np.max(digitized)
  mode=st.mode(digitized[np.nonzero(digitized)])
  mean=np.mean(digitized[np.nonzero(digitized)])
  new=np.zeros(x)
  prev_r=-1
  PR_interval=50
  R_central_difference=max-mean
  #Finding R peaks
  for i in range(x):
    if i>0 and (digitized[i]-digitized[i-1])>R_central_difference and (prev_r==-1 or (i-prev_r)>=PR_interval):
      new[i]=1
      prev_r=i
  #Plotting
  plt.rcParams['figure.dpi']=100
  f,axs=plt.subplots(3,1,figsize=(x*px, 6*y*px))
  axs[0].margins(x=0)
  axs[0].plot(range(len(new)),new)
  axs[0].set_title('Digital ECG')
  axs[1].imshow(t_img)
  axs[1].set_title('Thresholded Image')
  axs[2].imshow(imgr)
  axs[2].set_title('Original Image')
  plt.show()
  return new

In [ ]:
img=cv.imread('/content/tridevdeoghar_icu_mon--4_2023_1_4_9_20_1.jpeg')
image= hr_wave_detection(img)
cv.imwrite('hrwave_img.jpeg',image)

In [ ]:
arr=digitalECG(image)